In [1]:
import requests
import geopandas as gpd
import pandas as pd
import json

def download_geoboundaries(country_iso, level):
    """
    Download country shapefiles from GeoBoundaries API using a fixed URL template and load into memory.

    Parameters:
        country_iso (str): The ISO code of the country to download boundaries for.
        admin_level (str): The administrative level to download (e.g., "ADM0", "ADM1").

    Returns:
        geopandas.GeoDataFrame: GeoDataFrame containing the downloaded boundaries.
    """
    # URL template for GeoBoundaries API
    url_template = f"https://www.geoboundaries.org/api/current/gbOpen/{country_iso}/ADM{level}/"

    # Download the GeoJSON file
    response = requests.get(url_template)
    if response.status_code != 200:
        raise Exception(f"Failed to download GeoJSON for {country_iso} at level {level}. Status code: {response.status_code}")
    
    path_geojson = json.loads(response.content)["gjDownloadURL"]

    # Load the GeoJSON content into a GeoDataFrame
    gdf = gpd.read_file(path_geojson)

    gdf["GID_0"] = country_iso
    for l in range(level):
        gdf[f"GID_{l+1}"] = gdf[f"GID_{l}"] + "." + gdf.index.astype(str)

    print(f"GeoJSON boundary for {country_iso} (level {level}) loaded into memory")
    return gdf

def create_combined_gpkg(layers, country_iso, output_file="boundaries.gpkg"):
    """
    Create a GPKG file with combined ADM0 and ADM1 boundaries for a given country ISO code.

    Parameters:
        country_iso (str): The ISO code of the country to download boundaries for.
        output_file (str): Path to the output GPKG file.

    Returns:
        None
    """
    for layer in layers:
        gdfl = download_geoboundaries(country_iso, layer)
        gdfl.to_file(output_file, layer=f"ADM_ADM_{layer}", driver="GPKG")

    print(f"GPKG file created with combined boundaries at {output_file}")

# Example usage
if __name__ == "__main__":
    country_iso = "BWA"  # Replace with your desired country ISO code
    output_file = f"gadm41_{country_iso}.gpkg"  # Replace with your desired output file name

    create_combined_gpkg([0,1], country_iso, output_file)

GeoJSON boundary for BWA (level 0) loaded into memory
GeoJSON boundary for BWA (level 1) loaded into memory
GPKG file created with combined boundaries at gadm41_BWA.gpkg


In [ ]:
url_template = f"https://www.geoboundaries.org/api/current/gbOpen/NGA/ADM1/"

# Download the GeoJSON file
response = requests.get(url_template)

path_geojson = json.loads(response.content)["gjDownloadURL"]

# Load the GeoJSON content into a GeoDataFrame
gdf = gpd.read_file(path_geojson)

: 

: 

: 